
#**SC3000 Lab Assignment 1**
## Group Members :


1.   Kuan Hong Hui Joy U2221261D
2.   Tan Pin Yee U2221027H
3.   Cel XXX Matrix number



#**Setting up the enviroment**
1. Installing Neccessary Library
2. Importing Neccessary Library
3. Set up a virtual enviroment (not done yet)




In [2]:
import gym
import numpy as np
import math
import time
import matplotlib.pyplot as plt
import keras
import random

from random import shuffle
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import load_model
from collections import deque


from gym import logger as gymlogger
from gym.wrappers import RecordVideo
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data=''''''.format(encoded.decode('ascii'))))
  else:
    print("Could not find video")

##**Tutorial: Loading the CartPole environment**


In [3]:
# Creating the CartPole environment
env = gym.make("CartPole-v1")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
#prints the action space of the environment
print(env.action_space)

# printing the min and max values of the 4 observed values
print(env.observation_space)

Discrete(2)
Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


In [5]:
observation = env.reset()
print("Initial observations:", observation)

Initial observations: [-0.03191902  0.03472922 -0.01207161  0.02816264]


In [6]:
# 1st step taken in the environment
observation, reward, done, info = env.step(0)
print("New observations after choosing action 0:", observation)
print("Reward for this step:", reward)
print("Is this round done?", done)

New observations after choosing action 0: [-0.03122443 -0.16021755 -0.01150836  0.31701252]
Reward for this step: 1.0
Is this round done? False


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [7]:
# Cumulative reward from using naive strategy
observation = env.reset()
cumulative_reward = 0
done = False
while not done:
    observation, reward, done, info = env.step(0)
    cumulative_reward += reward
print("Cumulative reward for this round:", cumulative_reward)

Cumulative reward for this round: 8.0


# **Task 1: Development of an RL agent**


*   Test out the different RL
*   Training the model



## **Method 1 : Q learning**

In [8]:
#Define parameters
LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 6000 #
EPS = 1 # Epsilon
EPS_DEC = 0.99995 #Epsilon decay value

#define observation space discretization
Observation = [30, 30, 50, 50]
np_array_win_size = np.array([0.25, 0.25, 0.01, 0.1])

In [9]:
#Initialize Q-table with random values
q_table = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))

#Funciton to get the discrete state from the continuous state
def get_discrete_state(state):
    discrete_state = state / np_array_win_size + np.array([15, 10, 1, 10])
    return tuple(discrete_state.astype(int))


In [10]:
# Training RL Agent
for episode in range(EPISODES + 1):

    # Reset environment
    currentState = env.reset()

    discrete_state = get_discrete_state(currentState)
    done = False
    ep_reward = 0

    #Epsilon-greedy policy
    while not done:
        if np.random.random() > EPS:
            action = np.argmax(q_table[discrete_state])
        else:
            action = np.random.randint(0, env.action_space.n)

        # Step action
        new_state, reward, done, _ = env.step(action)
        ep_reward += reward
        new_discrete_state = get_discrete_state(new_state)

        #Updating Q-table
        if not done:
            max_future_q = np.max(q_table[new_discrete_state])
            current_q = q_table[discrete_state + (action,)]
            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
            q_table[discrete_state + (action,)] = new_q

        discrete_state = new_discrete_state

    #Decay epsilon
    if EPS > 0.05:
        EPS = max(EPS * EPS_DEC, 0.05)

env.close()

In [11]:
currentState = env.reset()
if np.random.random() > EPS:
    action = np.argmax(q_table[discrete_state])
else:
    action = np.random.randint(0, env.action_space.n)

print("Observation: ", currentState)
print("Action: ", action)

env.close()

Observation:  [ 0.04088416  0.0064101  -0.04834092 -0.04500221]
Action:  0


## **Method 2 : DQN**


https://github.com/VBot2410/Deep-Q-Learning-Cartpole/blob/master/Cartpole%20Deep%20Q-Learning.ipynb

In [12]:
ACTIONS_DIM = 2  ## Output Dimension=No. of possible actions (2)
OBSERVATIONS_DIM = 4  ## Input Dimension= No of Elements in State Tuple (4)
MAX_ITERATIONS = 500  ## Max Time Steps Per Game (Limited to 500 by Environment)
LEARNING_RATE = 0.001
NUM_EPOCHS = 50

GAMMA = 0.99  ## Future Reward Discount Factor
REPLAY_MEMORY_SIZE = 4000  ## Replay Memory Size
NUM_EPISODES = 500  ## Games Played in Training Phase
MINIBATCH_SIZE = 64  ## Number of Samples chosen randomly from Replay Memory

RANDOM_ACTION_DECAY = 0.99  ## The factor by which Random Action Probability Decreases
INITIAL_RANDOM_ACTION = 1  ## Initial Random Action Factor
Samples=[]  ## A list to store Individual Game Scores
Means=[]  ## A list to store Mean Score over Last 20 Games


In [13]:
class ReplayBuffer():
    def __init__(self, max_size):
        self.max_size = max_size
        self.transitions = deque()

    def add(self, observation, action, reward, observation2):
        if len(self.transitions) > self.max_size:
            if np.random.random() < 0.5:
                shuffle(self.transitions)
            self.transitions.popleft()
        self.transitions.append((observation, action, reward, observation2))

    def sample(self, count):
        return random.sample(self.transitions, count)

    def size(self):
        return len(self.transitions)

In [14]:
def get_q(model, observation):  ## Function to Predict Q-Values from the Model
    np_obs = np.reshape(observation, [-1, OBSERVATIONS_DIM])  ## Reshape the state
    return model.predict(np_obs,verbose=0)  ## Query the Model for possible actions and corresponding Q-Values

def train(model, observations, targets):  ## Function to Train the Model
    np_obs = np.reshape(observations, [-1, OBSERVATIONS_DIM])  ## Reshape the State
    np_targets = np.reshape(targets, [-1, ACTIONS_DIM])  ## Reshape the Target

    model.fit(np_obs, np_targets, epochs=1, verbose=0)  ## Fit the model using State-Target Pairs

def predict(model, observation):  ## Function to Predict Q-Values from Model
    np_obs = np.reshape(observation, [-1, OBSERVATIONS_DIM])  ## Reshape the State
    return model.predict(np_obs,verbose=0)  ## Query the Model for possible actions and corresponding Q-Values



def get_model():  ## Deep Q-Network
    model = Sequential()

    model.add(Dense(24, input_shape=(OBSERVATIONS_DIM, ), activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(2, activation='linear'))

    model.compile(
        optimizer=Adam(lr=LEARNING_RATE),  ## Adam Optimizer with Initial Learning Rate=0.001
        loss='mse',
        metrics=[],
    )

    return model



def update_action(action_model, target_model, sample_transitions):
    random.shuffle(sample_transitions)  ## Randomly Shuffle the Minibatch Samples
    batch_observations = []
    batch_targets = []

    for sample_transition in sample_transitions:  ## For each sample in Minibatch
        old_observation, action, reward, observation = sample_transition

        targets = np.reshape(get_q(action_model, old_observation), ACTIONS_DIM)
        targets[action] = reward  ## Set Target Value
        if observation is not None:  ## If observation is not Empty

            ## Query the Model for possible actions and corresponding Q-Values
            predictions = predict(target_model, observation)
            new_action = np.argmax(predictions)  ## Select the Best Action (Max Q-Value)

            ## Update the Target with Future Reward Discount Factor
            targets[action] += GAMMA * predictions[0, new_action]

        batch_observations.append(old_observation)  ## Add Old State to observations batch
        batch_targets.append(targets)  ## Add target to targets batch

    ## Update the model using Observations and their corresponding Targets
    train(action_model, batch_observations, batch_targets)


In [ ]:
## contains the training and testing results, so maybe can split the testing phase into task 2
##also the training takes damn long, maybe close to an hr
def main():
    Temp=[]  ## to hold most recent 100 Game Scores
    iteration=0  ## Initialize Time Step Number to Avoid initial No variable Error
    ## Set Random Action Probability to Initial Number(=1)
    random_action_probability = INITIAL_RANDOM_ACTION

    replay = ReplayBuffer(REPLAY_MEMORY_SIZE)  ## Initialize Replay Memory & Specify Maximum Capacity

    action_model = get_model()  ## Initialize action-value model with random weights

    env = gym.make('CartPole-v1')  ## Prepare the OpenAI Cartpole-v1 Environment

    for episode in range(NUM_EPISODES):  ## For Games 0 to Maximum Games Limit
        if np.mean(Temp) > 495 and iteration > 495: ## Training the model until it passes
            print('Passed')
            break

        ## If mean over the last 100 Games is >495, then Success!!!
        if np.mean(Temp)>495 and iteration>495:
            print('Passed')
            break

        ## Reduce the Random Action Probability by Decay Factor
        random_action_probability *= RANDOM_ACTION_DECAY
        observation = env.reset()

        for iteration in range(MAX_ITERATIONS):  ## Timesteps
            ## Generate Random Action Probability
            random_action_probability = max(random_action_probability, 0.1)
            old_observation = observation
            ## If generated fraction<Random Action Probability
            if np.random.random() < random_action_probability:
                ## Take Random Action (Explore)
                action = np.random.choice(range(ACTIONS_DIM))
            else:  ## If generated fraction>Random Action Probability
                ## Query the Model and Get Q-Values for possible actions
                q_values = get_q(action_model, observation)
                action = np.argmax(q_values)  ## Select the Best Action using Q-Values received
            ## Take the Selected Action and Observe Next State
            observation, reward, done, info = env.step(action)

            if done:
                Samples.append(iteration+1)  ## Add Final Score of the Game to the Scores List
                ## If Number of Games>100, Calculate Mean Over 100 Games to Check Convergence
                if len(Samples)>100:
                    Temp=Samples[-100:]  ## Select Score of Most Recent 100 Games
                    ## Add Mean of Most Recent 20 Games to a list
                    Means.append(np.mean(Samples[-20:]))
                ## Print End-Of-Game Information
                print(('Episode:{}, iterations:{}, RAP:{}').format(
                        episode,
                        iteration,random_action_probability))

                if iteration!=499:
                    reward = -5  ## Give -5 Reward for Taking Wrong Action Leading to Failure
                if iteration==499:
                    reward= 5  ## Give +5 Reward for Completing the Game Successfully
                ## Add the Observation to Replay Memory
                replay.add(old_observation, action, reward, None)
                break  ## Break and Start a new Game

            ## Add the Observation to Replay Memory
            replay.add(old_observation, action, reward, observation)

            ##  Update the Deep Q-Network Model
            if replay.size() >= MINIBATCH_SIZE and np.random.random()<0.25 and Samples[-1]<495:
                sample_transitions = replay.sample(MINIBATCH_SIZE)
                update_action(action_model, action_model, sample_transitions)


####
##  Here, the Training Phase Ends. Now We plot the Training Results and Save the Trained Model
##  We Also Test the Model for Another 100 Games.
####

    print("Training Process")
    plt.plot(Samples)
    plt.title('Training Phase')
    plt.ylabel('Cumulative Rewards')
    plt.ylim(ymax=510)
    plt.xlabel('Episodes')
    plt.savefig('Training.png', bbox_inches='tight')
    plt.show()


## can remove this graph
    plt.plot(Means)
    plt.title('Mean Score of Last 20 Games')
    plt.ylabel('Time Steps')
    plt.ylim(ymax=510)
    plt.xlabel('Trial')
    plt.savefig('Training_Average.png', bbox_inches='tight')
    plt.show()
    action_model.save('cartpole_model.h5')  ## Save the Trained Model
    del action_model
    action_model = load_model('cartpole_model.h5')  ## Load the Trained Model


## Here We test the trained model for 100 more games
    Tests=[]
    for i in range(0,100):
        observation = env.reset()
        while(True):
            old_observation = observation
            q_values = get_q(action_model, observation)
            action = np.argmax(q_values)
            observation, reward, done, info = env.step(action)
            if done:
                Tests.append(iteration+1)
                env.reset()
                break
    print(np.mean(Tests))
    plt.plot(Tests)
    plt.title('Testing Phase')
    plt.ylabel('Time Steps')
    plt.ylim(ymax=510)
    plt.xlabel('Trial')
    plt.savefig('Testing.png', bbox_inches='tight')
    plt.show()




if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Episode:0, iterations:23, RAP:0.99
Episode:1, iterations:24, RAP:0.9801
Episode:2, iterations:23, RAP:0.9702989999999999
Episode:3, iterations:19, RAP:0.96059601


#**Task 2: Demonstrate the effectiveness of the RL Agent**
*   Test the trained models against 100 episodes
*   Plot the graph for each method
*   Make comparision

## **Method 1 : Q-learning**
Running the trained Q-learning agent with 100 episode and plotting its cumulative rewards onto a graph to see the effectiveness of the training

In [ ]:
# Testing the agent
num_episodes = 100
max_time_steps = 500
rewards = []

for i in range(num_episodes):
    state = env.reset()
    discrete_state = get_discrete_state(state)
    total_reward = 0

    for t in range(max_time_steps):
        action = np.argmax(q_table[discrete_state])
        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        state = next_state
        discrete_state = get_discrete_state(state)

        if done:
            break

    #Printing results of each episode
    rewards.append(total_reward)
    print('Episode {}: Cumulative Reward = {}'.format(i+1, total_reward))

env.close()

In [ ]:
# Plot cumulative rewards
plt.plot(rewards)
plt.title('Cumulative reward for each episode')
plt.ylabel('Cumulative reward')
plt.xlabel('Episode')
plt.show()

# Calculate average cumulative reward
average_reward = sum(rewards) / num_episodes
print("Average cumulative reward:", average_reward)
print("Is my agent good enough?", average_reward > 195)


## **Method 2 : DQN**

# Task 3: Render one episode

In [ ]:

env = RecordVideo(gym.make("CartPole-v1"), "./video")
observation = env.reset()
while True:
    env.render()
    #your agent goes here
    action = rand_policy_agent(observation)
    observation, reward, done, info = env.step(action)



    if done:
      break;
env.close()
show_video()